<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CST3512/blob/main/ISQS2340_SQLite_mailbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ISQS2340 SQLite

**Introduction to Management Information Technology**
## Database
*with SQL, SQLite, and DB Browser*

### The Charles Severance mboxshort.txt example in SQLite

Slight modifications:
1. Changed variable name 'row' to 'record' in all
2. Added /sample_data/ path to file name

Import required libraries

In [10]:
import sqlite3

Connect to (create) the database and instantiate a variable for the SQLite cursor in the open database.    

*Note that this will happen in the volatile **content/sample_data** area of this Colab session.*

In [11]:
conn = sqlite3.connect('emaildb.sqlite') # connecting to the database file
cur = conn.cursor()  # opening a handle (variable) for the db cursor

Use the open database's cursor handle to drop the email table if it exists

In [ ]:
cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('''
CREATE TABLE Counts (email TEXT, count INTEGER)''')

The following code loops through the mbox-short.txt file, record-by-record.   

Be sure to have mbox-short.txt or whatever file you choose to read, available in the **content/sample_data** area of this Colab session. 

In [ ]:
fname = input('Enter file name: ')  # a valid file name in the Content area 
if (len(fname) < 1): fname = 'sample_data/mbox-short.txt' # default file name
fh = open(fname)  # open a file handle with the specified file
for line in fh:   # loop through the file, line-by-line
    if not line.startswith('From: '): continue
    pieces = line.split()
    email = pieces[1]
    # now use the open database as we would have used a dictionary
    cur.execute('SELECT count FROM Counts WHERE email = ? ', (email,))
    record = cur.fetchone()
    if record is None:
        cur.execute('''INSERT INTO Counts (email, count)
                VALUES (?, 1)''', (email,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE email = ?',
                    (email,))
        
    conn.commit() # commit the data for this record/statement to the db
    # the commit does not necessarily have to be done on every iteration
    # because commit is time consuming, it might be staggered

*After the loop above completes, the records of the mbox-short.txt file should be exhausted and the **emaildb** SQLite database should be populated with email addresses.*

Now query the database for the results.

In [ ]:
# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT email, count FROM Counts ORDER BY count DESC LIMIT 10'

for record in cur.execute(sqlstr):
    print(str(record[0]), record[1])

Don't forget to close the open db cursor and be nice to the database.

In [9]:
cur.close()



---

